# Wikipedia Knowledge Graph Builder - Anderson Database

This notebook creates a knowledge graph from Wikipedia pages by importing JSON data into a Neo4j database.
The data contains Wikipedia entities with their titles, URLs, and relationships to other entities.

**Database:** anderson (instead of matheus)
**Features:** Includes comprehensive test queries to verify database connectivity and data integrity.

## 1. Install Required Libraries

First, let's install the necessary libraries in our virtual environment.

In [2]:
# Install required libraries
!pip install neo4j pandas groq

import logging

# Option 1: Silence httpx completely
logging.getLogger("httpx").setLevel(logging.WARNING)


## 2. Import Libraries and Setup

Import the necessary libraries for data processing and Neo4j connectivity.

In [3]:
import json
import pandas as pd
from neo4j import GraphDatabase
import logging
from typing import List, Dict, Any
import time

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("Libraries imported successfully!")

Libraries imported successfully!


## 3. Configure Neo4j Connection

Set up the connection parameters for the Neo4j database.
**Note:** This connects to the 'anderson' database instead of 'matheus'.

In [4]:
from dotenv import load_dotenv, find_dotenv
import os

# Load environment variables from .env file
load_dotenv(find_dotenv())

# Neo4j connection details from environment variables
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
MY_GROQ_API_KEY = os.getenv("MY_GROQ_API_KEY")

In [5]:
NEO4J_DATABASE = "anderson"  # Changed from 'matheus' to 'anderson'

# Test connection
try:
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    driver.verify_connectivity()
    print(f"✅ Successfully connected to Neo4j database: {NEO4J_DATABASE}!")
except Exception as e:
    print(f"❌ Failed to connect to Neo4j: {e}")
    print("Please check your connection parameters and network access.")

✅ Successfully connected to Neo4j database: anderson!


## 4. Database Connection Tests

Run comprehensive tests to verify the database connection and basic functionality.

In [6]:
def run_connection_tests(driver):
    """
    Run comprehensive tests to verify database connectivity and basic functionality.
    """
    tests = {
        "Node Count": "MATCH (n) RETURN count(n) as total_nodes",
        "Relationship Count": "MATCH ()-[r]->() RETURN count(r) as total_relationships",
        "Node Examples": "MATCH (n) RETURN n LIMIT 3",
        "Relationship Examples": "MATCH p=()-->() RETURN p LIMIT 1"

    }
    
    print(f"🧪 Running connection tests for database: {NEO4J_DATABASE}")
    print("=" * 60)
    
    with driver.session(database=NEO4J_DATABASE) as session:
        for test_name, query in tests.items():
            try:
                print(f"\n🔍 {test_name}:")
                result = session.run(query)
                records = list(result)
                
                if records:
                    for record in records:
                        print(f"   {dict(record)}")
                else:
                    print("   No results returned")
                    
                print("   ✅ PASSED")
                
            except Exception as e:
                print(f"   ❌ FAILED: {e}")
                
        print("\n" + "=" * 60)
        print("🎯 Connection tests completed!")

# Run connection tests
run_connection_tests(driver)

🧪 Running connection tests for database: anderson

🔍 Node Count:
   {'total_nodes': 108071}
   ✅ PASSED

🔍 Relationship Count:
   {'total_relationships': 5122983}
   ✅ PASSED

🔍 Node Examples:
   {'n': <Node element_id='4:272e7fb9-be88-49e0-938d-4859e141ff40:0' labels=frozenset({'Document'}) properties={'document_title': 'Therefore sign', 'title_encode': 'Therefore_sign', 'document_url': 'https://en.wikipedia.org//w/index.php?title=Therefore_sign&amp;oldid=815234923'}>}
   {'n': <Node element_id='4:272e7fb9-be88-49e0-938d-4859e141ff40:1' labels=frozenset({'Document'}) properties={'document_title': 'Watchman (law enforcement)', 'title_encode': 'Watchman_(law_enforcement)', 'document_url': 'https://en.wikipedia.org//w/index.php?title=Watchman_(law_enforcement)&amp;oldid=807971712'}>}
   {'n': <Node element_id='4:272e7fb9-be88-49e0-938d-4859e141ff40:3' labels=frozenset({'Document'}) properties={'document_title': 'Super Bowl 50 halftime show', 'title_encode': 'Super_Bowl_50_halftime_show',

In [15]:
from groq import Groq
client = Groq(api_key=MY_GROQ_API_KEY)
# MODEL = "llama-3.3-70b-versatile"
MODEL = "llama-3.1-8b-instant"

def ask_LLM(question: str, model: str=MODEL) -> str:
    """
    Queries the Groq API with a question and model.

    Args:
        question (str): The input question for the model.
        model (str): The model to query.

    Returns:
        str: The response from the model as a string or None if an error occurs.
    """
    try:
        response = client.chat.completions.create(
            messages=[{"role": "user", "content": question}],
            model=model,
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error querying Groq API: {e}")
        return None

# Example usage
response = ask_LLM("What is Neo4j?")
print("Response from Groq API:", response)

Response from Groq API: **Neo4j** is a popular, open-source, graph database management system. It is designed to store and manage large amounts of graph data in an efficient and scalable manner.

### What is a Graph Database?

A graph database is a type of NoSQL database that stores data as a collection of nodes or vertices connected by edges. Each node represents an entity, and each edge represents a relationship between two entities. This structure allows for efficient querying and navigating of complex relationships between data entities.

### Key Features of Neo4j

1. **Graph Data Model**: Neo4j uses a native graph data model, allowing it to efficiently store and query graph data.
2. **ACID Compliance**: Neo4j supports Atomicity, Consistency, Isolation, and Durability (ACID) transactions, ensuring that database operations are executed reliably and safely.
3. **Schema-Agnostic**: Neo4j has a schema-agnostic design, allowing you to model your data without the need for a predefined sc

## 5. LLM-Guided Graph Search Algorithm

Implementation of a sophisticated graph search algorithm that uses LLM guidance for entity extraction, path pruning, and sufficiency checking.

In [8]:
def llm_extract_entities(query: str) -> List[str]:
    """
    Extract relevant entities from a query using LLM.
    
    Args:
        query (str): The input query
        
    Returns:
        List[str]: List of extracted entities
    """
    prompt = f"""
    Extract the main entities (people, places, concepts, organizations) from this query.
    Return only the entity names, one per line, without any additional text or formatting.
    
    Query: {query}
    
    Entities:
    """
    
    response = ask_LLM(prompt)
    if not response:
        return []
    
    # Parse entities from response
    entities = [entity.strip() for entity in response.split('\n') if entity.strip()]
    return entities

# Test the function
test_query = "What is the relationship between Albert Einstein and the theory of relativity?"
extracted_entities = llm_extract_entities(test_query)
print(f"Query: {test_query}")
print(f"Extracted entities: {extracted_entities}")

Query: What is the relationship between Albert Einstein and the theory of relativity?
Extracted entities: ['Albert Einstein', 'theory of relativity']


In [9]:
import random

def find_similar_nodes(entities: List[str], top_k: int = 5) -> List[Dict]:
    """
    Find nodes in the knowledge graph that match the extracted entities.
    Uses simple string matching for now (can be improved with embeddings later).
    
    Args:
        entities (List[str]): List of entity names to search for
        top_k (int): Number of top nodes to return
        
    Returns:
        List[Dict]: List of matching nodes with their properties
    """
    # Reconnect to the database
    driver_local = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    
    matching_nodes = []
    
    with driver_local.session(database=NEO4J_DATABASE) as session:
        for entity in entities:
            # Search for nodes with title containing the entity (case-insensitive)
            query = """
            MATCH (n)
            WHERE toLower(n.document_title) CONTAINS toLower($entity)
            RETURN n.document_title as title, n.document_url as url, n
            LIMIT $limit
            """
            
            result = session.run(query, entity=entity, limit=top_k)
            for record in result:
                node_data = {
                    'title': record['title'],
                    'url': record['url'],
                    'node': record['n'],
                    'search_entity': entity
                }
                matching_nodes.append(node_data)
    
    driver_local.close()
    
    # Return top_k unique nodes (remove duplicates)
    seen_titles = set()
    unique_nodes = []
    for node in matching_nodes:
        if node['title'] not in seen_titles:
            seen_titles.add(node['title'])
            unique_nodes.append(node)
    
    # randomly select top_k nodes if more than top_k found
    if len(unique_nodes) > top_k:
        unique_nodes = random.sample(unique_nodes, top_k)
    
    return unique_nodes

# Test the function
test_entities = ["Einstein", "relativity"]
similar_nodes = find_similar_nodes(test_entities, top_k=5)
print(f"Entities: {test_entities}")
print(f"Found {len(similar_nodes)} similar nodes:")
for i, node in enumerate(similar_nodes, 1):
    print(f"  {i}. {node['title']} (matched: {node['search_entity']})")

Entities: ['Einstein', 'relativity']
Found 5 similar nodes:
  1. Einstein–Szilárd letter (matched: Einstein)
  2. Theory of relativity (matched: relativity)
  3. Religious and philosophical views of Albert Einstein (matched: Einstein)
  4. General relativity (matched: relativity)
  5. Einstein field equations (matched: Einstein)


In [10]:
import math

def initialize_paths(initial_nodes: List[Dict]) -> List[List[Dict]]:
    """
    Initialize paths with the retrieved initial nodes.
    
    Args:
        initial_nodes (List[Dict]): List of initial nodes
        
    Returns:
        List[List[Dict]]: List of paths, each containing one initial node
    """
    paths = []
    for node in initial_nodes:
        path = [{'title': node['title'], 'url': node['url'], 'type': 'node'}]
        paths.append(path)
    return paths

def expand_paths(paths: List[List[Dict]], max_neighbors: int = 5) -> List[List[Dict]]:
    """
    Expand each path by one hop, finding successors and predecessors.
    
    Args:
        paths (List[List[Dict]]): Current paths
        max_neighbors (int): Maximum neighbors to explore per node
        
    Returns:
        List[List[Dict]]: Expanded paths
    """
    driver_local = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    expanded_paths = []
    
    with driver_local.session(database=NEO4J_DATABASE) as session:
        for path in paths:
            last_node = path[-1]
            last_node_title = last_node['title']
            
            # Get nodes already in the path to avoid cycles
            path_titles = {node['title'] for node in path if node['type'] == 'node'}
            
            # Find outgoing relationships
            query_out = """
            MATCH (a {document_title: $title})-[r]->(b)
            WHERE NOT b.document_title IN $path_titles
            RETURN type(r) as rel_type, b.document_title as target_title, b.document_url as target_url
            LIMIT $limit
            """
            
            # Find incoming relationships
            query_in = """
            MATCH (a)-[r]->(b {document_title: $title})
            WHERE NOT a.document_title IN $path_titles
            RETURN type(r) as rel_type, a.document_title as source_title, a.document_url as source_url
            LIMIT $limit
            """
            
            # Execute outgoing relationships
            result_out = session.run(query_out, title=last_node_title, 
                                   path_titles=list(path_titles), limit=1000)
            
            # Randomly sample max_neighbors from result_out
            result_out = list(result_out)
            if len(result_out) > max_neighbors:
                result_out = random.sample(result_out, math.ceil(max_neighbors / 2))

            for record in result_out:
                new_path = path.copy()
                new_path.append({
                    'type': 'relationship',
                    'rel_type': record['rel_type'],
                    'direction': 'outgoing'
                })
                new_path.append({
                    'type': 'node',
                    'title': record['target_title'],
                    'url': record['target_url']
                })
                expanded_paths.append(new_path)
            
            # Execute incoming relationships
            result_in = session.run(query_in, title=last_node_title, 
                                  path_titles=list(path_titles), limit=1000)
            result_in = list(result_in)
            if len(result_in) > max_neighbors:
                result_in = random.sample(result_in, math.floor(max_neighbors / 2))

            for record in result_in:
                new_path = path.copy()
                new_path.append({
                    'type': 'relationship',
                    'rel_type': record['rel_type'],
                    'direction': 'incoming'
                })
                new_path.append({
                    'type': 'node',
                    'title': record['source_title'],
                    'url': record['source_url']
                })
                expanded_paths.append(new_path)
    
    driver_local.close()
    return expanded_paths if expanded_paths else paths

# Test path initialization and expansion
test_nodes = [{'title': 'Test Node 1', 'url': 'url1'}, {'title': 'Test Node 2', 'url': 'url2'}]
initial_paths = initialize_paths(test_nodes)
print(f"Initialized {len(initial_paths)} paths:")
for i, path in enumerate(initial_paths, 1):
    print(f"  Path {i}: {[node['title'] for node in path if node['type'] == 'node']}")

# Test with real nodes (if we found any)
if similar_nodes:
    real_paths = initialize_paths(similar_nodes)
    expanded = expand_paths(real_paths, max_neighbors=2)

    print(f"\nExpanded to {len(expanded)} paths:")
    for i, path in enumerate(expanded, 1):  # Show first 3 paths
        if path[1]['direction'] == 'outgoing':
            direction = "->"
        else:
            direction = "<-"
        path_str = " {} ".format(direction).join([
            item['title'] if item['type'] == 'node' 
            else f"[{item['rel_type']}]" 
            for item in path
        ])
        print(f"  Path {i}: {path_str}")

Initialized 2 paths:
  Path 1: ['Test Node 1']
  Path 2: ['Test Node 2']

Expanded to 10 paths:
  Path 1: Einstein–Szilárd letter -> [CITES] -> Uranium-238
  Path 2: Einstein–Szilárd letter <- [CITES] <- History of nuclear weapons
  Path 3: Theory of relativity -> [CITES] -> Force
  Path 4: Theory of relativity <- [CITES] <- Viscosity
  Path 5: Religious and philosophical views of Albert Einstein -> [CITES] -> Causality
  Path 6: Religious and philosophical views of Albert Einstein <- [CITES] <- Brownian motion
  Path 7: General relativity -> [CITES] -> Angular momentum
  Path 8: General relativity <- [CITES] <- Earth science
  Path 9: Einstein field equations -> [CITES] -> Black hole
  Path 10: Einstein field equations <- [CITES] <- Timeline of quantum mechanics


In [12]:
def extract_path_triples(path: List[Dict]) -> List[str]:
    """
    Extract triples from a path representation.
    
    Args:
        path (List[Dict]): Path containing nodes and relationships
        
    Returns:
        List[str]: List of triples in string format
    """
    triples = []
    
    for i in range(0, len(path) - 2, 2):  # Step by 2 to get node-rel-node patterns
        if (i + 2 < len(path) and 
            path[i]['type'] == 'node' and 
            path[i + 1]['type'] == 'relationship' and 
            path[i + 2]['type'] == 'node'):
            
            source = path[i]['title']
            relation = path[i + 1]['rel_type']
            target = path[i + 2]['title']
            direction = path[i + 1]['direction']
            
            if direction == 'outgoing':
                triple = f"({source}) -[{relation}]-> ({target})"
            else:
                triple = f"({target}) -[{relation}]-> ({source})"
            
            triples.append(triple)
    
    return triples

def llm_score_path_relevance(query: str, path_triples: List[str]) -> float:
    """
    Score the relevance of a path to the query using LLM.
    
    Args:
        query (str): The original query
        path_triples (List[str]): List of triples from the path
        
    Returns:
        float: Relevance score between 0 and 1
    """
    if not path_triples:
        return 0.0
    
    triples_text = "\n".join(path_triples)
    
    prompt = f"""
    Rate the relevance of these knowledge graph triples to answering the given query.
    Return only a number between 0 and 1, where:
    - 0 = completely irrelevant
    - 1 = highly relevant and useful for answering the query
    
    Query: {query}
    
    Triples:
    {triples_text}
    
    Relevance score:
    """
    
    response = ask_LLM(prompt)
    if not response:
        return 0.0
    
    try:
        # Extract number from response
        score_str = response.strip().split()[0]
        score = float(score_str)
        return max(0.0, min(1.0, score))  # Clamp between 0 and 1
    except (ValueError, IndexError):
        return 0.0

def prune_paths(query: str, paths: List[List[Dict]], top_n_paths: int = 5) -> List[List[Dict]]:
    """
    Prune paths by scoring their relevance and keeping only the top N.
    
    Args:
        query (str): The original query
        paths (List[List[Dict]]): List of paths to prune
        top_n_paths (int): Number of top paths to keep
        
    Returns:
        List[List[Dict]]: Pruned paths sorted by relevance
    """
    if not paths:
        return []
    
    scored_paths = []
    
    for path in paths:
        triples = extract_path_triples(path)
        score = llm_score_path_relevance(query, triples)
        scored_paths.append((score, path))
    
    # Sort by score (descending) and return top N
    scored_paths.sort(key=lambda x: x[0], reverse=True)
    return [path for score, path in scored_paths[:top_n_paths]]

# Test triple extraction
test_path = [
    {'type': 'node', 'title': 'Albert Einstein'},
    {'type': 'relationship', 'rel_type': 'DEVELOPED', 'direction': 'outgoing'},
    {'type': 'node', 'title': 'Theory of Relativity'}
]

triples = extract_path_triples(test_path)
print(f"Test path triples: {triples}")

# Test path scoring
test_query = "What did Einstein develop?"
if triples:
    score = llm_score_path_relevance(test_query, triples)
    print(f"Relevance score for '{test_query}': {score}")

Test path triples: ['(Albert Einstein) -[DEVELOPED]-> (Theory of Relativity)']
Relevance score for 'What did Einstein develop?': 1.0


In [13]:
def llm_check_sufficiency(query: str, combined_triples: List[str]) -> bool:
    """
    Check if the combined triples are sufficient to answer the query.
    
    Args:
        query (str): The original query
        combined_triples (List[str]): Combined triples from all paths
        
    Returns:
        bool: True if sufficient, False otherwise
    """
    if not combined_triples:
        return False
    
    triples_text = "\n".join(combined_triples)
    
    prompt = f"""
    Given the following knowledge graph triples, can you answer this query completely and accurately?
    Answer only "Yes" or "No".
    
    Query: {query}
    
    Available triples:
    {triples_text}
    
    Can this query be answered with the given information?
    """
    
    response = ask_LLM(prompt)
    if not response:
        return False
    
    return response.strip().lower().startswith('yes')

def llm_generate_answer(query: str, final_triples: List[str]) -> str:
    """
    Generate the final answer using the selected triples.
    
    Args:
        query (str): The original query
        final_triples (List[str]): Final set of triples to use for answering
        
    Returns:
        str: Generated answer
    """
    if not final_triples:
        return "I don't have enough information to answer this query."
    
    triples_text = "\n".join(final_triples)
    
    prompt = f"""
    Based on the following knowledge graph triples, provide a comprehensive answer to the query.
    Use only the information provided in the triples. Be factual and concise.
    
    Query: {query}
    
    Available knowledge:
    {triples_text}
    
    Answer:
    """
    
    response = ask_LLM(prompt)
    return response if response else "Unable to generate an answer."

def combine_triples_from_paths(paths: List[List[Dict]]) -> List[str]:
    """
    Combine all triples from multiple paths, removing duplicates.
    
    Args:
        paths (List[List[Dict]]): List of paths
        
    Returns:
        List[str]: Combined unique triples
    """
    all_triples = set()
    
    for path in paths:
        triples = extract_path_triples(path)
        all_triples.update(triples)
    
    return list(all_triples)

# Test sufficiency checking
test_triples = [
    "(Albert Einstein) -[DEVELOPED]-> (Theory of Relativity)",
    "(Theory of Relativity) -[PUBLISHED_IN]-> (1905)"
]

test_query = "What did Einstein develop?"
is_sufficient = llm_check_sufficiency(test_query, test_triples)
print(f"Query: {test_query}")
print(f"Triples: {test_triples}")
print(f"Sufficient: {is_sufficient}")

# Test answer generation
if is_sufficient:
    answer = llm_generate_answer(test_query, test_triples)
    print(f"Generated answer: {answer}")

Query: What did Einstein develop?
Triples: ['(Albert Einstein) -[DEVELOPED]-> (Theory of Relativity)', '(Theory of Relativity) -[PUBLISHED_IN]-> (1905)']
Sufficient: True
Generated answer: Einstein developed the Theory of Relativity.


In [16]:
def llm_guided_search(query: str, top_k: int = 5, max_depth: int = 3, 
                     top_n_paths: int = 10, max_neighbors: int = 5) -> str:
    """
    Main LLM-guided graph search algorithm.
    
    Args:
        query (str): The input query
        top_k (int): Number of initial nodes to retrieve
        max_depth (int): Maximum search depth
        top_n_paths (int): Number of paths to keep after pruning
        max_neighbors (int): Maximum neighbors to explore per node
        
    Returns:
        str: Generated answer
    """
    print(f"🔍 Starting LLM-guided search for: '{query}'")
    
    # Step 1: Extract entities from query
    print("📝 Step 1: Extracting entities...")
    entities = llm_extract_entities(query)
    print(f"   Extracted entities: {entities}")
    
    if not entities:
        return "Could not extract any entities from the query."
    
    # Step 2: Retrieve initial nodes
    print("🎯 Step 2: Finding initial nodes...")
    initial_nodes = find_similar_nodes(entities, top_k)
    print(f"   Found {len(initial_nodes)} initial nodes")
    
    if not initial_nodes:
        return "Could not find any relevant nodes in the knowledge graph."
    
    # Step 3: Initialize paths
    print("🛤️ Step 3: Initializing paths...")
    paths = initialize_paths(initial_nodes)
    print(f"   Initialized {len(paths)} paths")
    
    # Step 4: Iterative search loop
    for depth in range(max_depth):
        print(f"\n🔄 Iteration {depth + 1}/{max_depth}")
        
        # SEARCH: Expand paths by one hop
        print("   🌐 Expanding paths...")
        old_path_count = len(paths)
        paths = expand_paths(paths, max_neighbors)
        print(f"   Expanded from {old_path_count} to {len(paths)} paths")
        
        if not paths:
            break
        
        # PRUNE: Keep only top N paths
        print("   ✂️ Pruning paths...")
        paths = prune_paths(query, paths, top_n_paths)
        print(f"   Kept {len(paths)} top paths")
        
        # REASONING: Check sufficiency
        print("   🧠 Checking sufficiency...")
        combined_triples = combine_triples_from_paths(paths)
        is_sufficient = llm_check_sufficiency(query, combined_triples)
        print(f"   Sufficient: {is_sufficient}")
        
        if is_sufficient:
            print("   ✅ Sufficiency achieved! Stopping search.")
            break
    
    # Step 5: Generate final answer
    print("\n📋 Step 5: Generating final answer...")
    final_triples = combine_triples_from_paths(paths)
    answer = llm_generate_answer(query, final_triples)
    
    print(f"✅ Search completed!")
    print(f"📊 Final statistics:")
    print(f"   - Entities found: {len(entities)}")
    print(f"   - Initial nodes: {len(initial_nodes)}")
    print(f"   - Final paths: {len(paths)}")
    print(f"   - Final triples: {len(final_triples)}")
    
    return answer

# Test the complete algorithm with a simple query
test_query = "What is the relationship between Albert Einstein and physics?"
print("🚀 Testing the complete LLM-guided search algorithm:")
print("=" * 80)

try:
    result = llm_guided_search(test_query, top_k=3, max_depth=2, top_n_paths=5)
    print(f"\n🎯 FINAL ANSWER:")
    print(result)
except Exception as e:
    print(f"❌ Error during search: {e}")
    import traceback
    traceback.print_exc()

🚀 Testing the complete LLM-guided search algorithm:
🔍 Starting LLM-guided search for: 'What is the relationship between Albert Einstein and physics?'
📝 Step 1: Extracting entities...
   Extracted entities: ['Albert Einstein', 'physics']
🎯 Step 2: Finding initial nodes...
   Found 3 initial nodes
🛤️ Step 3: Initializing paths...
   Initialized 3 paths

🔄 Iteration 1/2
   🌐 Expanding paths...
   Expanded from 3 to 16 paths
   ✂️ Pruning paths...
   Kept 5 top paths
   🧠 Checking sufficiency...
   Sufficient: True
   ✅ Sufficiency achieved! Stopping search.

📋 Step 5: Generating final answer...
✅ Search completed!
📊 Final statistics:
   - Entities found: 2
   - Initial nodes: 3
   - Final paths: 5
   - Final triples: 5

🎯 FINAL ANSWER:
Based on the available knowledge, there is a relationship between Albert Einstein and physics. However, Albert Einstein is not explicitly mentioned in the given knowledge.

Given the context of the available knowledge, it can be inferred that the relationsh